<a href="https://colab.research.google.com/github/Prattoypaul/deeplearning/blob/machinel/Sign_detection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import pandas as pd
import os,cv2
import matplotlib.pyplot as plt
import seaborn as sns
import tensorflow as tf
from tensorflow import keras


In [ ]:
dataset='/content/drive/MyDrive/Colab Notebooks/A==4 chatbot/Data set'

In [ ]:
df=keras.preprocessing.image_dataset_from_directory(
    dataset,seed=123,shuffle=True,image_size=(256,256),batch_size=32
)

In [ ]:
classes=df.class_names
classes

In [ ]:
len(dataset)

In [ ]:
for img,label in df.take(1):
  print(img.shape)
  print(label.numpy())

In [ ]:
len(dataset)*0.8

In [ ]:
train_ds=df.take(48)
test=df.skip(48)
test_ds=test.take(6)
val_ds=test.skip(6)

In [ ]:
train_ds=train_ds.cache().shuffle(100000).prefetch(buffer_size=tf.data.AUTOTUNE)
test_ds=test_ds.cache().shuffle(100000).prefetch(buffer_size=tf.data.AUTOTUNE)
val_ds=val_ds.cache().shuffle(100000).prefetch(buffer_size=tf.data.AUTOTUNE)

In [ ]:
plt.figure(figsize=(15,15))
for img,lab in df.take(1):
  for i in range(12):
    plt.subplot(3,4,i+1)
    plt.imshow(img[i].numpy().astype('uint8'))
    plt.title(classes[lab[i]])
    plt.axis('off')

In [ ]:
resize_rescall=keras.Sequential([
    keras.layers.experimental.preprocessing.Rescaling(1/255),
    keras.layers.experimental.preprocessing.Resizing(256,256)
])

In [ ]:
data_aug=keras.Sequential([
    keras.layers.experimental.preprocessing.RandomFlip('horizontal_and_vertical'),
    keras.layers.experimental.preprocessing.RandomRotation(0.3)
])

In [ ]:
from tensorflow.python.data.ops.dataset_ops import AUTOTUNE
from tensorflow.python import training
train_ds=train_ds.map(
    lambda x,y :(data_aug(x,training=True),y)
).prefetch(buffer_size=AUTOTUNE)

In [ ]:
input_shape=(32,256,256,3)

In [ ]:
model=keras.models.Sequential([
                               resize_rescall,
                               keras.layers.Conv2D(32,(3,3),activation='relu',input_shape=input_shape),
                               keras.layers.MaxPooling2D((2,2)),
                               keras.layers.Conv2D(64,(3,3),activation='relu'),
                               keras.layers.MaxPooling2D((2,2)),
                               keras.layers.Conv2D(264,(3,3),activation='relu'),
                               keras.layers.MaxPooling2D((2,2)),
                               keras.layers.Conv2D(512,(3,3),activation='relu'),
                               keras.layers.MaxPooling2D((2,2)),
                               keras.layers.Conv2D(512,(3,3),activation='relu'),
                               keras.layers.MaxPooling2D((2,2)),
                               keras.layers.Conv2D(1024,(3,3),activation='relu'),
                               keras.layers.MaxPooling2D((2,2)),
                               keras.layers.Flatten(),
                               keras.layers.Dense(1028,activation='relu'),
                               keras.layers.Dense(512,activation='relu'),
                               keras.layers.Dense(256,activation='relu'),
                               keras.layers.Dense(128,activation='relu'),
                               keras.layers.Dense(64,activation='relu'),
                              
                               keras.layers.Dense(21,activation='softmax')




                               
])

model.compile(
    optimizer='adam',
    loss=keras.losses.SparseCategoricalCrossentropy(from_logits=False),
    metrics=['acc']

)

In [ ]:
model.build(input_shape=input_shape)

In [ ]:
model.summary()

In [ ]:
ht=model.fit(train_ds,batch_size=32,validation_data=val_ds,verbose=1,
             epochs=50)

In [ ]:
model.evaluate(test_ds)

In [ ]:
ht.params

In [ ]:
len(ht.history['loss'])

In [ ]:
ht.history.keys()

In [ ]:
acc=ht.history['acc']
loss=ht.history['loss']
val_acc=ht.history['val_acc']
val_loss=ht.history['val_loss']

In [ ]:
plt.figure(figsize=(8,8))
plt.subplot(1,2,1)
plt.plot(range(50),acc,label='Traing_Accuracy')
plt.plot(range(50),val_acc,label='Validation_Accuracy')
plt.legend(loc='upper right')
plt.title("Training_Validation_Accuracy")

plt.subplot(1,2,2)
plt.plot(range(50),loss,label='Training_loss')
plt.plot(range(50),val_loss,label='Validtion_loss')
plt.legend(loc="lower left")
plt.title("training_Validation_loss")

plt.show()

In [ ]:
for img_batch,lab_batch in test_ds.take(1):
  first_img=img_batch[0].numpy().astype('uint8')
  first_label=lab_batch[0].numpy()

  plt.imshow(first_img)
  print("Acvtual_label:",classes[first_label])
  batch_prediction=model.predict(img_batch)
  print(
      "Predict_label:",classes[np.argmax(batch_prediction[0])]
  )

In [ ]:
def predict(model, img):
    img_array = tf.keras.preprocessing.image.img_to_array(images[i].numpy())
    img_array = tf.expand_dims(img_array, 0)

    predictions = model.predict(img_array)

    predicted_class = classes[np.argmax(predictions[0])]
    confidence = round(100 * (np.max(predictions[0])), 2)
    return predicted_class, confidence

In [ ]:
plt.figure(figsize=(15, 15))
for images, labels in test_ds.take(1):
    for i in range(9):
        ax = plt.subplot(3, 3, i + 1)
        plt.imshow(images[i].numpy().astype("uint8"))
        
        predicted_class, confidence = predict(model, images[i].numpy())
        actual_class = classes[labels[i]] 
        
        plt.title(f"Actual: {actual_class},\n Predicted: {predicted_class}.\n Confidence: {confidence}%")
        
        plt.axis("off")